Importing libraries

In [1]:
import numpy as np
import tensorflow as  tf
import pandas as pd
import matplotlib.pyplot as plt
import random

Collocation point creation

In [52]:
n_c = 5000  #collocation points

def Rand(start, end, num): 
    res = [] 
  
    for j in range(num): 
        res.append(random.uniform(start, end)) 
  
    return res


x_c = Rand(-1, 1, n_c)
t_c = Rand(0, 1, n_c)
x_c = np.array(x_c ,dtype='float64')
t_c = np.array(t_c ,dtype='float64')
input_c = np.vstack((x_c, t_c))
input_c = input_c.T


Architecture ANN

Prepocessing the data

In [54]:
pinn1 = tf.keras.models.Sequential()
pinn1.add(tf.keras.layers.Dense(2,activation='tanh'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh'))

pinn1.add(tf.keras.layers.Dense(1,activation='tanh'))         #output layer

Custom Loss function

In [58]:
def custom_loss_1(arch, x, t):
  def loss():
    with tf.GradientTape() as t1:
      with tf.GradientTape() as t2:
        xt = tf.stack((x_c, t_c), axis=1)
        u = arch(xt)
        u_x = t2.gradient(u, x)
        u_t = t2.gradient(u, t)
      u_xx = t1.gradient(u_x, x)
    f = u_t + u*u_x - (0.01/np.pi)*u_xx
    loss_res = tf.reduce_mean(tf.square(f))
    return loss_res
  return loss


loss_fn = custom_loss_1(pinn1, x_c, t_c)
pinn1.compile(optimizer='adam', loss=loss_fn, metrics=['mse'])
pinn1.fit(input_c, batch_size=20 ,epochs=1000)



Epoch 1/1000


ValueError: ignored

Test case

In [ ]:
x_test = np.array(random.uniform(-1, 1), dtype='float64')
t_test = np.array(random.uniform(0, 1), dtype='float64')
xt_test = tf.stack((x_test, t_test), axis=1)
u = pinn1(xt_test)